In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)
import warnings
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor 

from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

In [6]:
data = pd.read_csv("country_vaccinations.csv")
data.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


In [7]:
data.groupby('country').count()

,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
country,,,,,,,,,,,,,,
Afghanistan,351,351,40,37,30,3,350,40,37,30,350,351,351,351
Albania,394,394,236,183,173,168,393,236,183,173,393,394,394,394
Algeria,346,346,21,21,17,3,345,21,21,17,345,346,346,346
Andorra,364,364,38,33,27,2,363,38,33,27,363,364,364,364
Angola,339,339,57,55,47,0,338,57,55,47,338,339,339,339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wales,395,395,390,390,390,388,394,390,390,390,394,395,395,395
Wallis and Futuna,302,302,37,37,32,0,301,37,37,32,301,302,302,302
Yemen,275,275,18,18,11,0,274,18,18,11,274,275,275,275


In [8]:
data.isna().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                     36419
people_vaccinated                      38455
people_fully_vaccinated                41007
daily_vaccinations_raw                 43680
daily_vaccinations                       324
total_vaccinations_per_hundred         36419
people_vaccinated_per_hundred          38455
people_fully_vaccinated_per_hundred    41007
daily_vaccinations_per_million           324
vaccines                                   0
source_name                                0
source_website                             0
dtype: int64

In [9]:
data = data.drop(data[data.total_vaccinations.isna()].index)
data= data.drop(data[data.people_vaccinated.isna()].index)
data.isna().sum()

country                                   0
iso_code                                  0
date                                      0
total_vaccinations                        0
people_vaccinated                         0
people_fully_vaccinated                3272
daily_vaccinations_raw                 6713
daily_vaccinations                      206
total_vaccinations_per_hundred            0
people_vaccinated_per_hundred             0
people_fully_vaccinated_per_hundred    3272
daily_vaccinations_per_million          206
vaccines                                  0
source_name                               0
source_website                            0
dtype: int64

In [10]:
difference = data.total_vaccinations.mean() - data.people_vaccinated.mean()
difference_per_hundred = data.total_vaccinations_per_hundred.mean() - data.people_vaccinated_per_hundred.mean()

data.people_vaccinated = data.people_vaccinated.fillna(data.total_vaccinations - difference)
data.people_vaccinated_per_hundred = data.people_vaccinated_per_hundred.fillna(data.total_vaccinations_per_hundred - difference_per_hundred)

data.isna().sum()

country                                   0
iso_code                                  0
date                                      0
total_vaccinations                        0
people_vaccinated                         0
people_fully_vaccinated                3272
daily_vaccinations_raw                 6713
daily_vaccinations                      206
total_vaccinations_per_hundred            0
people_vaccinated_per_hundred             0
people_fully_vaccinated_per_hundred    3272
daily_vaccinations_per_million          206
vaccines                                  0
source_name                               0
source_website                            0
dtype: int64

In [11]:
data.daily_vaccinations = data.daily_vaccinations.fillna(0)
data.daily_vaccinations_per_million = data.daily_vaccinations_per_million.fillna(0)
data.people_fully_vaccinated = data.people_fully_vaccinated.fillna(0)
data.people_fully_vaccinated_per_hundred = data.people_fully_vaccinated_per_hundred.fillna(0)
data.daily_vaccinations_raw = data.daily_vaccinations_raw.fillna(0) 

data.isna().sum()

country                                0
iso_code                               0
date                                   0
total_vaccinations                     0
people_vaccinated                      0
people_fully_vaccinated                0
daily_vaccinations_raw                 0
daily_vaccinations                     0
total_vaccinations_per_hundred         0
people_vaccinated_per_hundred          0
people_fully_vaccinated_per_hundred    0
daily_vaccinations_per_million         0
vaccines                               0
source_name                            0
source_website                         0
dtype: int64

In [12]:
NavidData = data.copy()

NavidData.rename(columns={'country':'Country',
                          'iso_code':'IsoCode',
                          'date':'Date',
                          'total_vaccinations': 'TotalVaccinations', 
                          'people_vaccinated': 'PeopleVaccinated',
                          'people_fully_vaccinated': 'PeopleFullyVaccinated',
                          'daily_vaccinations_raw': 'DailyVaccinationsRaw',
                          'daily_vaccinations': 'DailyVaccinations',
                          'total_vaccinations_per_hundred': 'TotalVaccinationsPerHundred',
                          'people_vaccinated_per_hundred':'PeopleVaccinatedPerHundred',
                          'people_fully_vaccinated_per_hundred': 'PeopleFullyVaccinatedPerHundred',
                          'daily_vaccinations_per_million': 'DailyVaccinationsPerMillion',
                          'vaccines':'Vaccines',
                          'source_name':'SourceName',
                          'source_website':'SourceWebsite'}, inplace=True) 

In [13]:
del NavidData['IsoCode']
del NavidData['Vaccines']
del NavidData['SourceName']
del NavidData['SourceWebsite']

In [14]:
Indonesia = NavidData[NavidData.Country.str.contains("Indonesia")]

In [15]:
scaler = StandardScaler()
Indonesia[['TotalVaccinations','PeopleVaccinated','PeopleFullyVaccinated','DailyVaccinationsRaw','DailyVaccinations','DailyVaccinations','TotalVaccinationsPerHundred','PeopleVaccinatedPerHundred','PeopleFullyVaccinatedPerHundred','DailyVaccinationsPerMillion']] = scaler.fit_transform(Indonesia[['TotalVaccinations','PeopleVaccinated','PeopleFullyVaccinated','DailyVaccinationsRaw','DailyVaccinations','DailyVaccinations','TotalVaccinationsPerHundred','PeopleVaccinatedPerHundred','PeopleFullyVaccinatedPerHundred','DailyVaccinationsPerMillion']])

In [19]:
IndonesiaX = Indonesia[['PeopleVaccinated','PeopleFullyVaccinated','DailyVaccinationsRaw','DailyVaccinations','DailyVaccinations','TotalVaccinationsPerHundred','PeopleVaccinatedPerHundred','PeopleFullyVaccinatedPerHundred','DailyVaccinationsPerMillion']]   
IndonesiaY = Indonesia[['TotalVaccinations']] #Target

In [20]:
X_train_I, X_test_I, Y_train_I, Y_test_I = train_test_split(IndonesiaX,IndonesiaY, test_size = 0.20, random_state =2,shuffle=False)

In [21]:
XTMindonesia = ExtraTreesRegressor()
XTMindonesia.fit(X_train_I, Y_train_I)

ExtraTreesRegressor()

In [22]:
Y_pred_I = XTMindonesia.predict(X_test_I)

In [24]:
print('MAE:',metrics.mean_absolute_error(Y_test_I, Y_pred_I))
print('MSE:',metrics.mean_squared_error(Y_test_I, Y_pred_I))
print('RMSE:',np.sqrt(metrics.mean_squared_error(Y_test_I, Y_pred_I)))

MAE: 0.621752609411571
MSE: 0.4956649857298686
RMSE: 0.7040347901417008
